# Lesson 3: Data Packaging
## 1. Tokenizing and creating input_ids

Start by loading the dataset from the previous lesson:

In [6]:
!pip install -U datasets transformers

In [7]:
#from datasets import load_dataset
#import pandas as pd

# Step 1: Load dataset
#ds = load_dataset("roneneldan/TinyStories", split="train")

In [8]:

# Step 2: Shuffle and select 500,000 stories
#sampled_ds = ds.shuffle(seed=42).select(range(500_000))

# Step 3: Convert to pandas DataFrame
#df = pd.DataFrame(sampled_ds)

# Step 4: Save to parquet
#df.to_parquet("tinystories_500k.parquet", index=False)

#print("Done! Saved 500,000 TinyStories to tinystories_500k.parquet")

In [9]:
import pandas as pd
import datasets

pd.set_option('display.max_colwidth', None)

In [72]:
df = pd.read_parquet("final_pretrain_ds.parquet")
df.sample(5)

text
31946                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [75]:
dataset = datasets.Dataset.from_pandas(df)
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 40457
})


In [76]:
from transformers import AutoTokenizer
model_path_or_name = "upstage/SOLAR-10.7B-v1.0"
tokenizer = AutoTokenizer.from_pretrained(
    model_path_or_name
)

In [80]:
tokens = tokenizer.tokenize("I love coding because")
tokens

['▁I', '▁love', '▁coding', '▁because']

In [81]:
tokenizer.convert_tokens_to_ids(tokens)

[315, 2016, 22363, 1096]

Create a helper function:

In [82]:
def tokenization(example):
    # Tokenize
    tokens = tokenizer.tokenize(example["text"])

    # Convert tokens to ids
    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    # Add <bos>, <eos> tokens to the front and back of tokens_ids
    # bos: begin of sequence, eos: end of sequence
    token_ids = [
        tokenizer.bos_token_id] \
        + token_ids \
        + [tokenizer.eos_token_id
    ]
    example["input_ids"] = token_ids

    # We will be using this column to count the total number of tokens
    # in the final dataset
    example["num_tokens"] = len(token_ids)
    return example

Tokenize all the examples in the pretraining dataset:

In [83]:
dataset = dataset.map(tokenization, load_from_cache_file=False)
print(dataset)

Map:   0%|          | 0/40457 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'num_tokens'],
    num_rows: 40457
})


In [84]:
sample = dataset[2]

print("text: ", sample["text"][:100]) #
print("\ninput_ids", sample["input_ids"][:30])
print("\nnum_tokens", sample["num_tokens"])

text:  Australia beats China, works toward delaying Team USA rematch until gold-medal game
By Dan FeldmanAu

input_ids [1, 6664, 347, 1449, 5077, 28725, 3791, 4112, 8579, 288, 6867, 7035, 1003, 855, 1996, 5014, 28733, 1591, 282, 2039, 13, 1930, 4294, 401, 4333, 1294, 21575, 28705, 28740, 28750]

num_tokens 358


Check the total number of tokens in the dataset:

In [85]:
import numpy as np
np.sum(dataset["num_tokens"])

np.int64(50684948)

## 2. Packing the data

Concatenate input_ids for all examples into a single list:

In [86]:
input_ids = np.concatenate(dataset["input_ids"])
print(len(input_ids))

50684948


In [87]:
max_seq_length = 32

In [88]:
total_length = len(input_ids) - len(input_ids) % max_seq_length
print(total_length)

50684928


Discard extra tokens from end of the list so number of tokens is exactly divisible by `max_seq_length`:

In [89]:
input_ids = input_ids[:total_length]
print(input_ids.shape)

(50684928,)


In [90]:
input_ids_reshaped = input_ids.reshape(-1, max_seq_length).astype(np.int32)
input_ids_reshaped.shape

(1583904, 32)

In [91]:
type(input_ids_reshaped)

numpy.ndarray

Convert to Hugging Face dataset:

In [92]:
input_ids_list = input_ids_reshaped.tolist()
packaged_pretrain_dataset = datasets.Dataset.from_dict(
    {"input_ids": input_ids_list}
)
print(packaged_pretrain_dataset)

Dataset({
    features: ['input_ids'],
    num_rows: 1583904
})


## 3. Save the packed dataset to disk

In [93]:
packaged_pretrain_dataset.to_parquet("pretrain_ds.parquet")

Creating parquet from Arrow format:   0%|          | 0/1584 [00:00<?, ?ba/s]

209075328